# Make Visual Words

## Rquirements

k means clustering is memory intensive process. 

## Input

* n-dim descriptors
    * For making visual words, which image contains which descriptor is not important. We just do quantization.

* Number of visual words

## Process

Build cluster.
Say, we use k-means clustering. We determine number of clusters. 
Run k-means clustering. 
We can accelerate speed by using approximate nearest neighbor search such as kd-tree, but we lose accuracy. 

## Output

index of visual words, and its cluster's center point. 

From a given feature, we can use the output information to determine most closest visual words. 



In [1]:
# Load all features in the dataset. 
# Q. What if there is excatly same feature many times? do we add it into feature pool? 
# Q. Does many same feature point affect the result of k-menas clustering?

# An image may contain several features, and we need to get all features. 


% load_ext autoreload
% autoreload 2
from utils.oxf5k_feature_reader import feature_reader

all_features = feature_reader()
print(len(all_features))
# gc.collect()

read features from ./data/feature/feat_oxc1_hesaff_sift.bin
16334970


In [2]:
% load_ext autoreload
% autoreload 2
from utils import clustering_utils
import numpy as np

dim = 128

# n = 80000
# k = 20000
# data_points = np.random.randint(256, size=(n, dim), dtype=np.uint8)

data_points = np.array(all_features)
k = 131072 # 2^17 = 131072

# Timing History:
# n = 80,000 k=20,000 => single k-d tree with FLANN, 13 step, 6 min 


print('data_points:', data_points)
print('data_points shape:', data_points.shape)
print('data_points dtype:', data_points.dtype)
print('k:', k)

max_val = 2**8-1 # 255. uint8 max. for SIFT descriptor
print('max_val:', max_val)
center_points = np.random.randint(max_val, size=(k, dim), dtype=np.uint8) 
# center_points = center_points.astype(float)
print('center_points:', center_points)
print('center_points type:', center_points.dtype)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
data_points: [[ 38  18   4 ...,  45  49  10]
 [110  50   4 ...,   4  44  43]
 [102  28   1 ...,  50 110   2]
 ..., 
 [ 65  83  58 ...,   0   0   3]
 [ 34  64  70 ...,   1   1  13]
 [ 22   8  36 ...,  27  12  13]]
data_points shape: (16334970, 128)
data_points dtype: uint8
k: 131072
max_val: 255
center_points: [[214  19 135 ..., 222  47 216]
 [ 69 182 213 ..., 137 149  36]
 [ 10   3 148 ..., 178 244 227]
 ..., 
 [208 239 149 ...,  80  95  56]
 [ 37  68 222 ..., 168  44  84]
 [195 189 118 ...,  70 152  32]]
center_points type: uint8


In [3]:
%%time

import random
import pqkmeans
import pickle

# Train a PQ encoder.
# Each vector is devided into 4 parts and each part is
# encoded with log256 = 8 bit, resulting in a 32 bit PQ code.
M=8
print('encode with subsapce: {}'.format(M))
encoder = pqkmeans.encoder.PQEncoder(num_subdim=M, Ks=256)
# Q. do we have to use only subset? If we have train/test set split, use train for this. use test for query
# encoder.fit(data_points[:1000])
# time complexity for PQ: O(DL) where D is number of points, L is ???

# num_to_select = 100000                     # set the number to select here. 100K for 16M

num_to_select = 1000000 # Usually, people make 1M vocab in early 2010s
# 16000000
selected_index = np.random.choice(data_points.shape[0], num_to_select, replace=False)
list_of_random_items = data_points[selected_index, :]
print('shape of rand sample:', list_of_random_items.shape)

save_file_path = 'pqencoder_{}k_random_sample_from_{}M.pkl'.format(num_to_select//1000, data_points.shape[0]//1000000)
print("save file path:", save_file_path)

print("fitting encoder...")
encoder.fit(list_of_random_items)  # Use top 1M descriptor for training visual words codebook for oxford5k
pickle.dump(encoder, open(save_file_path, 'wb'))
print("done")

# Timing. codebook learning of 128d 4sub 100k took 3 min
# Timing. codebook learning of 128d 4sub 1M took 30 min
# Timing. codebook learning of 128d 8sub 1M took 58 min

encode with subsapce: 8
shape of rand sample: (1000000, 128)
save file path: pqencoder_1000k_random_sample_from_16M.pkl
fitting encoder...
done
CPU times: user 29min 14s, sys: 1d 12h 18min 46s, total: 1d 12h 48min 1s
Wall time: 57min 36s


In [4]:
%%time
print('transform whole set')
# Convert input vectors to 32-bit PQ codes, where each PQ code consists of four uint8.
# You can train the encoder and transform the input vectors to PQ codes preliminary.
from tqdm import tqdm as tqdm

# # For big-data that cannot fit in memory, we can use generator
pqcode_generator = encoder.transform_generator(data_points)
N, _ = data_points.shape
data_points_pqcode = np.empty([N, M], dtype=encoder.code_dtype)
print("data_points_pqcode.shape:\n{}".format(data_points_pqcode.shape))
print("data_points_pqcode.nbytes:\n{} bytes".format(data_points_pqcode.nbytes))
print("data_points_pqcode.dtype:\n{}".format(data_points_pqcode.dtype))
for n, code in enumerate(tqdm(pqcode_generator, total=N)):
    data_points_pqcode[n, :] = code

# For small data fit in memory, simply run this. 
# data_points_pqcode = encoder.transform(data_points)

save_file_path = 'data_points_pqcode_with_{}k_codebook_{}_sub.npy'.format(num_to_select//1000, M)
print("save file path:", save_file_path)

np.save(save_file_path, data_points_pqcode)
print("done")

# Memory: With 32bit PQ code, 16M 128d descriptor takes 65339960 bytes = 62.3 MB

  0%|          | 0/16334970 [00:00<?, ?it/s]

transform whole set
data_points_pqcode.shape:
(16334970, 8)
data_points_pqcode.nbytes:
130679760 bytes
data_points_pqcode.dtype:
uint8


100%|██████████| 16334970/16334970 [03:55<00:00, 69291.46it/s]


save file path: data_points_pqcode_with_1000k_codebook_8_sub.npy
done
CPU times: user 29min 25s, sys: 2h 7min 22s, total: 2h 36min 48s
Wall time: 3min 55s


In [5]:
%%time
save_file_path = 'clustering_centers_numpy_{}M_feature_{}k_coodebook_{}_sub_{}k_cluster.npy'.format(data_points.shape[0]//1000000, num_to_select//1000, M, k//1000)
print("save file path:", save_file_path)

print('run k-means with k:', k)
kmeans = pqkmeans.clustering.PQKMeans(encoder=encoder, k=k)
clustered = kmeans.fit_predict(data_points_pqcode)

print('clustered len:', len(clustered))

clustering_centers_numpy = np.array(kmeans.cluster_centers_, dtype=encoder.code_dtype)  # Convert to np.array with the proper dtype
np.save(save_file_path, clustering_centers_numpy)# Then, clustered[0] is the id of assigned center for the first input PQ code (X_pqcode[0]).

# Timing: 16M features, 1M codebook with 4 subspaces, 131k cluster: 3h 39min
# Timing: 16M features, 1M codebook with 8 subspaces, 131k cluster: 5h 10min

save file path: clustering_centers_numpy_16M_feature_1000k_coodebook_8_sub_131k_cluster.npy
run k-means with k: 131072
clustered len: 16334970
CPU times: user 8d 12h 47min 28s, sys: 3min 34s, total: 8d 12h 51min 3s
Wall time: 5h 10min 3s


In [6]:
print(len(clustered))
print(clustered[:100])

16334970
[105031, 68426, 122039, 83601, 120423, 66520, 46672, 84148, 50034, 96736, 35462, 110366, 31576, 101061, 82376, 43444, 86647, 44124, 37170, 94394, 113728, 40562, 128085, 5021, 66410, 12129, 122921, 77915, 20192, 130256, 46154, 2664, 35714, 84516, 131045, 61253, 40068, 48473, 107411, 33163, 55187, 45048, 90057, 67174, 69707, 55895, 78043, 87026, 123114, 119128, 16212, 122787, 7118, 8133, 34621, 6291, 44602, 79164, 3581, 102205, 51221, 35374, 19250, 16128, 24437, 52062, 108034, 45757, 25169, 80047, 114605, 20558, 27225, 110869, 110752, 103082, 99896, 49008, 99177, 60879, 31801, 68656, 33775, 94363, 87216, 115390, 9619, 41610, 119355, 36939, 64928, 85471, 86170, 32955, 45586, 60633, 14970, 92233, 2007, 105419]


----------- Below codes are legacy ------------------

In [ ]:
%%time
clustering_utils.run_kmeans_clustering(data_points, k, init_center_points=center_points, knn_method="naive")

# Way to speed up codebook learning

* [Annoy]() to assign centroids for all features
* FLANN vs ANNOY


In [ ]:
%%time
center_points = clustering_utils.run_kmeans_clustering(data_points, k, max_iter=15, init_center_points=center_points, knn_method="kdtree")

k means clustering with k=131072, knn_method:kdtree
k-means clustering. start step:0
num_same_centroid: 0
k-means clustering. start step:1
num_same_centroid: 1070631
k-means clustering. start step:2
num_same_centroid: 6305761
k-means clustering. start step:3
num_same_centroid: 8634853
k-means clustering. start step:4
num_same_centroid: 9524823
k-means clustering. start step:5
num_same_centroid: 9975007
k-means clustering. start step:6
num_same_centroid: 10253542
k-means clustering. start step:7
num_same_centroid: 10525599
k-means clustering. start step:8
num_same_centroid: 10715520
k-means clustering. start step:9
num_same_centroid: 10820592
k-means clustering. start step:10
num_same_centroid: 10881514
k-means clustering. start step:11
num_same_centroid: 10961578
k-means clustering. start step:12
num_same_centroid: 11051719
k-means clustering. start step:13
num_same_centroid: 11078194
k-means clustering. start step:14
num_same_centroid: 11135441
k-means clustering. start step:15
num_sa

In [111]:
%%time
clustering_utils.run_kmeans_clustering(data_points.astype(float), k, init_center_points=center_points, knn_method="kdtree")

k means clustering with k=20000, knn_method:kdtree
k-means clustering. start step:0
num_same_centroid: 0
k-means clustering. start step:1
num_same_centroid: 74836
k-means clustering. start step:2
num_same_centroid: 78470
k-means clustering. start step:3
num_same_centroid: 79362
k-means clustering. start step:4
num_same_centroid: 79690
k-means clustering. start step:5
num_same_centroid: 79870
k-means clustering. start step:6
num_same_centroid: 79931
k-means clustering. start step:7
num_same_centroid: 79974
k-means clustering. start step:8
num_same_centroid: 79985
k-means clustering. start step:9
num_same_centroid: 79995
k-means clustering. start step:10
num_same_centroid: 80000
cluster assignment is not changed. It means convergence!
CPU times: user 47min 37s, sys: 4.91 s, total: 47min 42s
Wall time: 7min 47s


In [24]:
%%time
clustering_utils.run_kmeans_clustering(data_points, k, init_center_points=center_points, knn_method="kdtree")

k means clustering with k=20000, knn_method:kdtree
k-means clustering. start step:0
num_same_centroid: 0
k-means clustering. start step:1
num_same_centroid: 74767
k-means clustering. start step:2
num_same_centroid: 78475
k-means clustering. start step:3
num_same_centroid: 79350
k-means clustering. start step:4
num_same_centroid: 79707
k-means clustering. start step:5
num_same_centroid: 79870
k-means clustering. start step:6
num_same_centroid: 79921
k-means clustering. start step:7
num_same_centroid: 79954
k-means clustering. start step:8
num_same_centroid: 79980
k-means clustering. start step:9
num_same_centroid: 79993
k-means clustering. start step:10
num_same_centroid: 79994
k-means clustering. start step:11
num_same_centroid: 79997
k-means clustering. start step:12
num_same_centroid: 79999
k-means clustering. start step:13
num_same_centroid: 80000
cluster assignment is not changed. It means convergence!
CPU times: user 36min 40s, sys: 4.51 s, total: 36min 44s
Wall time: 6min 2s
